INFO:tensorflow:Restoring parameters from /home/ubuntu/my_test_model-1000


In [1]:
! yes y | %reset
! yes y |conda install -c conda-forge gensim 

/bin/sh: 1: %reset: not found
yes: standard output: Broken pipe
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.7

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

yes: standard output: Broken pipe


In [2]:
import os, sys
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
import re

In [3]:
###Use this cell later if you have to read test files form directory 
##Currently, don't use this , just load test fiels name directly from test.csv file
def read_data(foldername):
    filenames=[]
    for fname in os.listdir(foldername):
        filenames.append(fname)
        
    return filenames
existing_files = read_data('/home/ubuntu/Static')
existingfilenames_df = pd.DataFrame(np.array(existing_files).reshape(len(existing_files),1), columns = ['apk'])
existingfilenames_df.head()
fileswithlabel = pd.read_csv("/home/ubuntu/labelledfileswith-txt.txt")
fileswithlabel.head()
fileslistwithlabel = existingfilenames_df.set_index('apk').join(fileswithlabel.set_index('apk'))
del existingfilenames_df
fileslistwithlabel.reset_index(inplace=True)
del fileswithlabel
fileslistwithlabel.head()
fileslistwithlabel['label'].value_counts()


0    36088
1    11963
Name: label, dtype: int64

In [4]:
save_strat_test_set = pd.read_csv("/home/ubuntu/test.csv")
save_strat_test_set.head()

,apk,label
0,AD0AC844EBAB732056C7B0FDB7A286DD5279047DBC8FA2...,0
1,7e1efb3c6fbf513e43e2444cc1ffd322.txt,1
2,AB78EFF2D53026C096B0E6E92EE039BB62A27537D30D6E...,0
3,12994c43fcd01e1a1d1b370101ab5c5a.txt,1
4,AE1562D977AC2811266ACDDDE2D846CE61BC37E5ECBCA5...,0


In [5]:
def TEST(session,split_size=5):
    validation_num=0
    which_fold=1
    X_test = save_strat_test_set['apk']  #training data features; currently just filenames
    y_test = save_strat_test_set['label']  #training data label
    acc,loss = run_validation_or_test(session,which_fold,X_test,y_test,True)
    print("Test Accuracy for the fold "+ str(which_fold) + " = "+ str(acc)+" loss = "+str(loss))   

In [6]:
batch_size=300
def run_validation_or_test(session,which_fold,X_train,y_train,isTest=False):
    batch_accuracy=[]
    batch_loss=[]
    total_batch = int(len(X_train)/batch_size)
    #total_batch=1
    for i in range (total_batch):
        batch_x = X_train[i*batch_size:(i+1)*batch_size]
        batch_y = y_train[i*batch_size:(i+1)*batch_size]
        #now we have the relevant filenames only
        batch_y = make_label(batch_y)
        sentences, _ = read_data("/home/ubuntu/Static/",batch_x)
        #print("read data")
        #nwo we need to convert sentences to word-embedded features and then we are ready to train
        batch_x = get_embedding(sentences)
        #print("got embedding")
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size,timesteps,num_input))
        # Run optimization op (backprop)
        loss,acc = session.run([loss_op,accuracy],feed_dict={xin: batch_x,Y: batch_y})
        print("Accuracy-acc = " +str(acc) + "benign  = "+str(batch_y[:,0].sum()) + "  malicious = "+str(batch_y[:,1].sum()))
        batch_accuracy.append(acc)
        batch_loss.append(loss)
        
        if isTest==True:
            pred_out=tf.argmax(prediction,1)
            arr = session.run(pred_out,feed_dict={xin: batch_x, Y: batch_y,learning_rate: 0.01})
            newarr = arr.tolist()
            from sklearn.metrics import confusion_matrix
            #print(batch_y[:10])
            #print(newarr[:10])
            one_dtestlabel= []
            for item in batch_y:
                if item[0]==0:
                    one_dtestlabel.append(1)
                else:
                    one_dtestlabel.append(0)
            cmatrix = confusion_matrix(one_dtestlabel,newarr)
            import matplotlib.pyplot as plt
            import itertools
            plt.figure()
            plot_confusion_matrix(cmatrix, classes=['Benign','Malicious'],title='Confusion matrix')
            figname = "test-Cmatrix"+str(i)+".png"
            plt.savefig(figname)
    return np.mean(batch_accuracy), np.mean(batch_loss)
    

In [7]:
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [8]:
def read_data(foldername,fileslist):
    sentences=[]
    numwords=[]
    for fname in fileslist:
        with open(os.path.join(foldername, fname)) as f:
            content = f.read().strip()
            content = re.sub(r"[\n#]","",content).split(" ")
            counter = len(content)
            numwords.append(counter)   #only consider the 4000 APIs# talk and decide later// part of tuning
        sentences.append(content[:4000])
        
    return sentences,numwords

def make_label(val):
    
    d = np.zeros((val.size,2))
    i = 0;
    for item in val:
        if item==0: #if benign
            d[i][0]=1.0
        else:
            d[i][1]=1.0
        i+=1
    return d
def get_embedding(sentences):
    d = np.zeros((len(sentences),4000,100))
    i=0
    for sentence in sentences:
        j=0
        for word in sentence:
            d[i][j]= model.wv[word]
            j+=1
        i+=1
    return d
            
    

In [9]:
from gensim.models import KeyedVectors
model = KeyedVectors.load("/home/ubuntu/model-all-100dim-cbag.kv", mmap='r')

In [10]:
training_steps = 500
              #means there are 84 batches in an epoch with each batch having 150 apps 
total_epochs = 20
display_step = 200
# Network Parameters
num_input = 100 # MNIST data input (img shape: 28*28)
timesteps = 4000 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 2 # Binary classification

In [11]:
sess1=tf.Session() 

graph = tf.get_default_graph()
weights = tf.Variable(tf.random_normal([num_hidden, num_classes]), dtype=tf.float32, name="weights")
bias = tf.Variable(tf.random_normal([num_classes]), dtype=tf.float32, name="bias")
init_op = tf.global_variables_initializer()
sess1.run(init_op)
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('/home/ubuntu/Newmodel_3.meta')
saver.restore(sess1,tf.train.latest_checkpoint('./'))


INFO:tensorflow:Restoring parameters from /home/ubuntu/Newmodel_3


In [12]:
xin = graph.get_tensor_by_name("placeholders/place_xin:0")
Y = graph.get_tensor_by_name("placeholders/place_y:0")
learning_rate = graph.get_tensor_by_name("place_learning:0")
 
#Now, access the op that you want to run. 
accuracy = graph.get_tensor_by_name("Accuracy/op_accuracy:0")
correct_pred = graph.get_tensor_by_name("evaluation/op_correct_pred:0")
loss_op = graph.get_tensor_by_name("Lossmeasure/op_loss:0")
prediction = graph.get_tensor_by_name("Softmax/op_prediction:0")

In [13]:
#tf.global_variables()
#All_varaibles = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

In [ ]:
TEST(sess1)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:31: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Accuracy-acc = 0.9633333benign  = 221.0  malicious = 79.0
Confusion matrix, without normalization
[[218   3]
 [  8  71]]
Accuracy-acc = 0.96benign  = 233.0  malicious = 67.0
Confusion matrix, without normalization
[[228   5]
 [  7  60]]
Accuracy-acc = 0.9633333benign  = 233.0  malicious = 67.0
Confusion matrix, without normalization
[[227   6]
 [  5  62]]
Accuracy-acc = 0.99benign  = 222.0  malicious = 78.0
Confusion matrix, without normalization
[[220   2]
 [  1  77]]
Accuracy-acc = 0.95666665benign  = 221.0  malicious = 79.0
